In [30]:
from src.logger import logging

logging.info("This is a test message")

In [2]:
import os
import sys
import traceback
import json
from dotenv import load_dotenv
import pandas as pd

# Add the root directory to the Python path
# sys.path.append(os.path.dirname(os.path.dirname(os.path.abspath(__file__))))

# from src.logger import logging
# from src.utlis import read_file, get_table_data

# Import Langchain dependencies
from langchain.chains import LLMChain, SequentialChain
from langchain.prompts import PromptTemplate
from langchain_openai.chat_models import ChatOpenAI

In [3]:
# Load environment variables
load_dotenv()
api_key = os.getenv("API_KEY")
base_url = os.getenv("BASE_URL")

print(api_key)

ddc-KEK18k3KBFlCjT9iEz0YjOUiWet7U0DRDdAKgHNgQ28MSXKVB9


In [4]:
# Initialize the OpenAI Chat model
llm = ChatOpenAI(
    openai_api_key=api_key,
    base_url=base_url,
    model_name='gpt-4o',
    temperature=0.9,
)


In [5]:
# Initialize the LLMChain with the OpenAI Chat model


llm = llm

template="""
Text:{text}
You are an expert MCQ maker. Given the above text, it is your job to \
create a quiz  of {number} multiple choice questions for {subject} students in {tone} tone. 
Make sure the questions are not repeated and check all the questions to be conforming the text as well.
Make sure to format your response like  RESPONSE_JSON below  and use it as a guide. \
Ensure to make {number} MCQs
### RESPONSE_JSON
{response_json}

"""

quiz_generation_prompt = PromptTemplate(
    input_variables=["text", "number", "subject", "tone", "response_json"],
    template=template)


quiz_chain=LLMChain(llm=llm,prompts=quiz_generation_prompt,output_key="quiz",verbose=True)


template2="""
You are an expert english grammarian and writer. Given a Multiple Choice Quiz for {subject} students.\
You need to evaluate the complexity of the question and give a complete analysis of the quiz. Only use at max 50 words for complexity analysis. 
if the quiz is not at per with the cognitive and analytical abilities of the students,\
update the quiz questions which needs to be changed and change the tone such that it perfectly fits the student abilities
Quiz_MCQs:
{quiz}

Check from an expert English Writer of the above quiz:
"""


quiz_evaluation_prompt=PromptTemplate(input_variables=["subject", "quiz"], template=template2)

review_chain=LLMChain(llm=llm, prompt=quiz_evaluation_prompt, output_key="review", verbose=True)


# This is an Overall Chain where we run the two chains in Sequence
generate_evaluate_chain=SequentialChain(chains=[quiz_chain, review_chain], input_variables=["text", "number", "subject", "tone", "response_json"],
                                        output_variables=["quiz", "review"], verbose=True,)
response = generate_evaluate_chain(
    {
        "text": "The quick brown fox jumps over the lazy dog",
        "number": 5,
        "subject": "English",
        "tone": "friendly",
        "response_json": {
            "question": "What is the color of the fox?",
            "options": ["brown", "black", "white", "red"],
            "answer": "brown"
        }
    }
)

print(response.get('quiz'))


C:\Users\SAJID HAMEED\AppData\Local\Temp\ipykernel_6932\1731957847.py:23: LangChainDeprecationWarning: The class `LLMChain` was deprecated in LangChain 0.1.17 and will be removed in 1.0. Use :meth:`~RunnableSequence, e.g., `prompt | llm`` instead.
  quiz_chain=LLMChain(llm=llm,prompts=quiz_generation_prompt,output_key="quiz",verbose=True)


ValidationError: 2 validation errors for LLMChain
prompt
  Field required [type=missing, input_value={'llm': ChatOpenAI(client...'quiz', 'verbose': True}, input_type=dict]
    For further information visit https://errors.pydantic.dev/2.10/v/missing
prompts
  Extra inputs are not permitted [type=extra_forbidden, input_value=PromptTemplate(input_vari...N\n{response_json}\n\n'), input_type=PromptTemplate]
    For further information visit https://errors.pydantic.dev/2.10/v/extra_forbidden